# MTH3302 - Méthodes probabilistes et statistiques pour I.A.
#### Polytechnique Montréal


### Projet A2025

----
### Contexte
Le béton est le matériau le plus important en génie civil. La résistance à la compression du béton est une fonction fortement non linéaire de son âge et de ses constituants. Ces constituants comprennent le ciment, le laitier de haut fourneau, les cendres volantes, l’eau, le superplastifiant, les gros granulats et les granulats fins.

Vous avez accès à une vraie banque de données répertoriant la résistance à la compression du béton en fonction de ses constituants. Ces données ont déjà été analysées avec des réseaux de neurones dans plusieurs articles scientifiques, notamment Chen (1998).

### Objectif
L'objectif principal est de développer un modèle statistique permettant la prédiction de la résistance à la compression (**Strength**) des 330 recettes de l'ensemble de test, en fonction de leurs caractéristiques et de leur âge.

### Description des Données
Les données sont réparties en deux fichiers :
- `train.csv` : 700 recettes avec la résistance connue (Ensemble d'entraînement).
- `test.csv` : 330 recettes dont la résistance est à prédire.

## Chargement des données
Importation des librairies nécessaires à l'analyse et à la modélisation.

In [1]:
using CSV, DataFrames, Statistics, Plots, LinearAlgebra, Random

default(size=(800, 500), margin=5Plots.mm)

### Chargement des données d'entraînement et de test
Nous chargeons les fichiers CSV dans des DataFrames pour faciliter la manipulation.

In [2]:
train_df = CSV.read("data/train.csv", DataFrame)
test_df = CSV.read("data/test.csv", DataFrame)

println("Dimensions Train : ", size(train_df))
println("Dimensions Test  : ", size(test_df))

first(train_df, 5)

Dimensions Train : (700, 10)
Dimensions Test  : (330, 9)


Row,ID,Cement,Slag,Ash,Water,Superplasticizer,Coarse,Fine,Age,Strength
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64
1,1,102.0,153.0,0.0,192.0,0.0,887.0,942.0,3,4.57
2,2,153.0,102.0,0.0,192.0,0.0,888.0,943.1,3,4.78
3,3,141.3,212.0,0.0,203.5,0.0,971.8,748.5,3,4.83
4,4,183.9,122.6,0.0,203.5,0.0,959.2,800.0,3,4.9
5,5,385.0,0.0,0.0,186.0,0.0,966.0,763.0,1,6.27


### Vérification de la qualité des données
Nous vérifions s'il existe des valeurs manquantes qui pourraient fausser l'analyse. Nous voyons qu'il n'y a aucune donnée manquante. (nmissing = 0 pour toutes les variables)

In [4]:
describe(train_df)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,ID,350.5,1,350.5,700,0,Int64
2,Cement,283.868,102.0,277.0,540.0,0,Float64
3,Slag,74.8664,0.0,24.0,359.4,0,Float64
4,Ash,53.7016,0.0,0.0,200.0,0,Float64
5,Water,181.207,121.8,185.0,246.9,0,Float64
6,Superplasticizer,6.24386,0.0,6.5,32.2,0,Float64
7,Coarse,973.72,801.0,968.0,1145.0,0,Float64
8,Fine,772.727,594.0,778.5,992.6,0,Float64
9,Age,44.9,1,28.0,365,0,Int64
